# Extract icExpBackground  in  OGA for Holograms

 work with Weakly_2023_01
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/02/04
- update : 2023/02/04


In [1]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2023_01/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-5.1.0/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2023_01/Spectractor 	setup


In [2]:
! eups list -s lsst_distrib

   g754a7f0350+935911ccad 	current w_2023_01 setup


In [3]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [4]:
import lsst.afw.math as afwMath
import lsst.daf.base as dafBase

In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits

import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [6]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [7]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [8]:
transform = AsinhStretch() + PercentileInterval(99.)

In [9]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None



## Configuration

In [10]:
#repo =  "/sdf/group/rubin/repo/main"
repo = "/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [11]:
DATE = 20230202

In [12]:
date_to_version_rebin1 = {
    20230118 : 'v13',
    20230117 : 'v14',
    20230119 : 'v15',
    20230131 : 'v16',
    20230201 : 'v17',
    20230202 : 'v18',
}

## Give the collection

In [13]:
#my_collection = "u/dagoret/BPS_test"
#my_collection = "u/dagoret/BPS_photimaging_202301"

#my_collection = "u/dagoret/BPS_test"
#my_collection = "u/dagoret/BPS_photimaging_202301"


#my_collection = "u/dagoret/BPS_manyspectro_v7" # january 20th 2023 (2023/01/17)
#my_collection = "u/dagoret/BPS_manyspectro_v8" # january 23th 2023 (2023/01/18)
#my_collection = "u/dagoret/BPS_manyspectro_v9" # january 23th 2023 (2023/01/19)
#my_collection = "u/dagoret/BPS_manyspectro_v10" # january 27th 2023 (2023/01/18), output with the _b, rebin=2
#my_collection = "u/dagoret/BPS_manyspectro_v11" # january 26th 2023 (2023/01/17), output with the _b, rebin=2
#my_collection = "u/dagoret/BPS_manyspectro_v12" # january 26th 2023 (2023/01/19), output with the _b, rebin=2,
#my_collection = "u/dagoret/BPS_manyspectro_v13" # january 31th 2023 (2023/01/18), rebin=1,
#my_collection = "u/dagoret/BPS_manyspectro_v14" # january 31th 2023 (2023/01/17), rebin=1,
#my_collection = "u/dagoret/BPS_manyspectro_v15" # january 31th 2023 (2023/01/18), rebin=1,
#my_collection = "u/dagoret/BPS_manyspectro_v16" # february 1st 2023 (2023/02/03), rebin=1,
#my_collection = "u/dagoret/BPS_manyspectro_v18" # february 2nd 2023 (2023/02/03), rebin=1,

the_version= date_to_version_rebin1[DATE] 
my_collection = f"u/dagoret/BPS_manyspectro_{the_version}"

In [14]:
file_icBgd_out = f"Table_icExpBackground_{DATE}.csv" 

## Config

In [15]:
# path index for each month
#DATE="20211103"
#filterdispersername = "empty~holo4_003"
#filterdispersername = "BG40~holo4_003"
#filterdispersername = "FELH0600~holo4_003"

# icExpBackground

In [16]:
datasetRefs = butler.registry.queryDatasets(datasetType='icExpBackground', collections=my_collection, where= "instrument='LATISS'")

In [17]:
df_icExpBg = pd.DataFrame(columns=['id','day_obs', 'seq_num','exposure','mjd','time','med','sig','N','band','physical_filter'])

In [18]:
all_ref_for_icExpBg  = []
all_dataId = []
all_exposures = []
all_icExpBg  = []
all_dateobs = []
all_mjd = []

for i, ref in enumerate(datasetRefs):
    print("========================datasetType = icExpBackground ============================================")
    print("fullId..................:",ref.dataId.full)
    print("visit...................:",ref.dataId["visit"])
    print("band....................:",ref.dataId["band"])
    print("physical filter.........:",ref.dataId["physical_filter"])
    print("run.....................:",ref.run)
    the_visit = ref.dataId["visit"]
    the_exposure = the_visit
    the_day_obs = ref.dataId["visit"]//100_000
    the_seq_num = ref.dataId["visit"]- the_day_obs*100_000    
    the_dataId = {'day_obs': the_day_obs,'seq_num':the_seq_num,'detector':0,}
    the_band=ref.dataId["band"]
    the_physical_filter = ref.dataId["physical_filter"]
    
    # extract time
    raw_img= butler.get('raw', dataId={'exposure': the_visit, 'instrument': 'LATISS', 'detector': 0}, collections = 'LATISS/raw/all')
    meta = raw_img.getMetadata()
    md = meta.toDict()
    date_obs = md['DATE-OBS']
    
    print(the_dataId)
    
    icExpBg = butler.get('icExpBackground', visit=the_exposure, detector=0, collections=my_collection, instrument='LATISS')
    all_ref_for_icExpBg.append(ref) 
    all_dataId.append(the_dataId) 
    all_exposures.append(the_exposure)
    all_icExpBg.append(icExpBg)
    
    outputStats = dafBase.PropertyList()

    # example of stat here : https://github.com/lsst/cp_pipe/blob/main/python/lsst/cp/pipe/cpFlatNormTask.py
    
    stats = afwMath.makeStatistics(icExpBg.getImage(),afwMath.MEANCLIP | afwMath.STDEVCLIP | afwMath.NPOINT)
    the_median = stats.getValue(afwMath.MEANCLIP)
    the_sigma = stats.getValue(afwMath.STDEVCLIP)
    the_N = stats.getValue(afwMath.NPOINT)
        
    print(the_median,the_sigma,the_N)
    
    the_visid_idx = i
    the_dateobs = pd.to_datetime(date_obs)
    the_mjd = Time(date_obs).mjd
    
    all_dataId.append(the_dataId) 
    all_dateobs.append(date_obs)
    all_mjd.append(the_mjd)
    
    #df_icExpBg = pd.DataFrame(columns=['id','day_obs', 'seq_num','exposure','mjd','time'])
    df_icExpBg.loc[i] = [i, the_day_obs, the_seq_num,the_exposure,the_mjd,the_dateobs,the_median,the_sigma,the_N,the_band,the_physical_filter]
                         
                        
    
    #registry_query = f"instrument='LATISS' and instrument=0 and visit={the_visit}]"
    #print(f"registry_query = {registry_query}")
    #info = registry.queryDimensionRecords('exposure',where = registry_query)
    
    #if i > 1:
    #    break

========================datasetType = icExpBackground ============================================
fullId..................: {band: 'white', instrument: 'LATISS', detector: 0, physical_filter: 'empty~holo4_003', visit: 2023020100632}
visit...................: 2023020100632
band....................: white
physical filter.........: empty~holo4_003
run.....................: u/dagoret/BPS_manyspectro_v18/20230203T114724Z
{'day_obs': 20230201, 'seq_num': 632, 'detector': 0}
65.73446412494842 13.000996829443807 16288000.0
========================datasetType = icExpBackground ============================================
fullId..................: {band: 'white', instrument: 'LATISS', detector: 0, physical_filter: 'empty~holo4_003', visit: 2023020100578}
visit...................: 2023020100578
band....................: white
physical filter.........: empty~holo4_003
run.....................: u/dagoret/BPS_manyspectro_v18/20230203T114724Z
{'day_obs': 20230201, 'seq_num': 578, 'detector': 0}
82.4

In [19]:
df_icExpBg

,id,day_obs,seq_num,exposure,mjd,time,med,sig,N,band,physical_filter
0,0,20230201,632,2023020100632,59977.219134,2023-02-02 05:15:33.172,65.734464,13.000997,16288000.0,white,empty~holo4_003
1,1,20230201,578,2023020100578,59977.192610,2023-02-02 04:37:21.526,82.450613,14.454371,16288000.0,white,empty~holo4_003
2,2,20230201,472,2023020100472,59977.138642,2023-02-02 03:19:38.681,46.162452,11.794052,16288000.0,white,empty~holo4_003
3,3,20230201,530,2023020100530,59977.168927,2023-02-02 04:03:15.271,82.068171,15.673638,16288000.0,white,empty~holo4_003
4,4,20230201,623,2023020100623,59977.214692,2023-02-02 05:09:09.431,86.455249,17.319804,16288000.0,white,empty~holo4_003
...,...,...,...,...,...,...,...,...,...,...,...
63,63,20230201,597,2023020100597,59977.200799,2023-02-02 04:49:09.076,77.584937,13.910326,16288000.0,white,empty~holo4_003
64,64,20230201,405,2023020100405,59977.106865,2023-02-02 02:33:53.163,52.422197,9.797952,16288000.0,white,empty~holo4_003
65,65,20230201,508,2023020100508,59977.157309,2023-02-02 03:46:31.492,54.023033,14.383790,16288000.0,white,empty~holo4_003
66,66,20230201,471,2023020100471,59977.138147,2023-02-02 03:18:55.864,46.294366,11.824970,16288000.0,white,empty~holo4_003


In [20]:
df_icExpBg.to_csv(file_icBgd_out)